## LangChain-對話(Chat)範例
2025/06/29 蘇彥庭

1. 單輪對話(Single-turn Chat)

In [2]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    HumanMessage("你是誰?"),
]

res = model.invoke(messages)

print(res.content)

我是一個大型語言模型，由 Google 訓練。我可以根據你的要求提供各種內容，例如：

*   **回答問題：** 試著回答你提出的問題，無論是關於知識、創意、或想像力方面。
*   **生成文本：** 寫不同類型的文本，比如詩歌、程式碼、 स्क्रिप्ट、音樂作品、郵件、信件等。
*   **翻譯語言：** 將文本從一種語言翻譯成另一種語言。
*   **總結文本：** 提取文本中的關鍵信息。
*   **重複文本：** 讓我知道要重複一次文本。

簡單來說，我是一個用語言來幫助你的人工智能。

你有什么想和我聊的吗？


2. 系統訊息控制(System Message Control)
* SystemMessage
    * 用途：提供系統角色設定或對模型行為的指示。
    * 常見用法：用來告訴模型應該扮演什麼樣的角色或遵循哪些規則。
* HumanMessage
    * 用途：模擬使用者的輸入。
    * 常見用法：用來傳遞使用者的問題或請求。
* AIMessage
    * 用途：模擬 AI 的回應。
    * 常見用法：用來記錄對話中 AI 所回覆的內容，讓後續對話有上下文。

In [3]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    SystemMessage("Translate the following from Chinese into English"),
    HumanMessage("你是誰?"),
]

res = model.invoke(messages)

print(res.content)

You are asking “Who are you?” in Chinese. 

The correct translation is: **Who are you?** or **You are who?**

It’s a straightforward question. 😊


3 具記憶的多輪對話(Multi-turn Chat with Memory)

3.1 多輪對話無記憶性範例

In [5]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    HumanMessage(content="請記住代號: A=狗 B=貓 C=鳥"),
]

res = model.invoke(messages)

print("\n---------- 第1次回答 ----------\n")
print(res.content)


messages = [
    HumanMessage(content="請問A, B, C分別代表什麼?")
]

res = model.invoke(messages)

print("\n---------- 第2次回答 ----------\n")
print(res.content)


---------- 第1次回答 ----------

好的，我記住了：

A=狗
B=貓
C=鳥

現在我準備好接收你的指令了。 😊


---------- 第2次回答 ----------

在上下文中，A, B, C 通常代表 **蘋果、番茄和車子**。 

這是一個常見的圖像謎語，需要根據上下文來推斷答案。

你提供的文字中沒有明確指出是哪個圖片，所以需要更多資訊才能判斷答案。

如果你能提供更多上下文，我可以更準確地回答你的問題。



3.2 對話具記憶性範例

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

model = init_chat_model("gemma3:1b", model_provider="ollama")

messages = [
    HumanMessage(content="請記住代號: A=狗 B=貓 C=鳥"),
    AIMessage(content="好，我記住了 A=狗 B=貓 C=鳥"),  # 模擬 AI 回應 幫助模型記住前一輪內容
    HumanMessage(content="請問A, B, C分別代表什麼?"),  # 新一輪輸入
]

res = model.invoke(messages)

print(res.content)

好的，根據你提供的代號，A代表**狗**，B代表**貓**，C代表**鳥**。

很高興能幫到你！ 😊



3.3 用 ConversationBufferMemory 建立帶記憶的 Chat 流程

In [7]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langchain.memory import ConversationBufferMemory

# 建立 Ollama 模型
llm = init_chat_model("gemma3:1b", model_provider="ollama")

# 建立記憶體
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history",
)

# 手動模擬對話輪次
def ask(input_text):
    # 載入記憶訊息
    history = memory.chat_memory.messages
    # 加上當前人類輸入
    messages = history + [HumanMessage(content=input_text)]
    # 呼叫模型
    response = llm.invoke(messages)
    # 將新對話加入記憶
    memory.chat_memory.add_user_message(input_text)
    memory.chat_memory.add_ai_message(response.content)
    return response.content

# 測試
print("\n---------- 第1次 ----------\n")
print(ask("請記住代號: A=狗 B=貓 C=鳥"))

print("\n---------- 第2次 ----------\n")
print(ask("請問A, B, C分別代表什麼?"))


C:\Users\User\AppData\Local\Temp\ipykernel_1184\1558259831.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(



---------- 第1次 ----------

好的，我記住了。

A=狗
B=貓
C=鳥

我現在能夠理解並使用代號 A、B、C 的意思。


---------- 第2次 ----------

A 代表 **狗**， B 代表 **貓**， C 代表 **鳥**。

重複確認，我理解了！ 😊



3.4 利用 Streamlit 快速搭一個對話服務

程式已寫好在`chat_app.py`，請執行`streamlit run chat_app.py`指令。